In [2]:
import numpy as np
import random as rnd
import torch
import time
import copy
import os
from torch.utils.tensorboard import SummaryWriter
from collections import deque
import shutil

In [3]:
import argparse
import os
import shutil
from random import random, randint, sample, seed

import numpy as np
import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter

import copy

from collections import deque

In [4]:
class Cube():
    
    def __init__(self):
        self.tensor = torch.tensor([torch.full([3,3],1).tolist(),
                     torch.full([3,3],2).tolist(),
                     torch.full([3,3],3).tolist(),
                     torch.full([3,3],4).tolist(),
                     torch.full([3,3],5).tolist(),
                     torch.full([3,3],6).tolist()])
        
        
    def R_(self):
        aux = torch.clone(self.tensor[0][:,2])
        self.tensor[0][:,2] = self.tensor[1][:,2]
        self.tensor[1][:,2] = self.tensor[2][:,2]
        self.tensor[2][:,2] = self.tensor[3][:,2]
        self.tensor[3][:,2] = aux
        
    def R(self):
        aux = torch.clone(self.tensor[0][:,2])
        self.tensor[0][:,2] = self.tensor[3][:,2]
        self.tensor[3][:,2] = self.tensor[2][:,2]
        self.tensor[2][:,2] = self.tensor[1][:,2]
        self.tensor[1][:,2] = aux
        
    def L(self):
        aux = torch.clone(self.tensor[0][:,0])
        self.tensor[0][:,0] = self.tensor[1][:,0]
        self.tensor[1][:,0] = self.tensor[2][:,0]
        self.tensor[2][:,0] = self.tensor[3][:,0]
        self.tensor[3][:,0] = aux
    
    def L_(self):
        aux = torch.clone(self.tensor[0][:,0])
        self.tensor[0][:,0] = self.tensor[3][:,0]
        self.tensor[3][:,0] = self.tensor[2][:,0]
        self.tensor[2][:,0] = self.tensor[1][:,0]
        self.tensor[1][:,0] = aux
    
    def U(self):
        aux = torch.clone(self.tensor[0][0,:])
        self.tensor[0][0,:] = self.tensor[4][0,:]
        self.tensor[4][0,:] = self.tensor[2][0,:]
        self.tensor[2][0,:] = self.tensor[5][0,:]
        self.tensor[5][0,:] = aux
        
    def U_(self):
        aux = torch.clone(self.tensor[0][0,:])
        self.tensor[0][0,:] = self.tensor[5][0,:]
        self.tensor[5][0,:] = self.tensor[2][0,:]
        self.tensor[2][0,:] = self.tensor[4][0,:]
        self.tensor[4][0,:] = aux
    
    def D(self):
        aux = torch.clone(self.tensor[0][2,:])
        self.tensor[0][2,:] = self.tensor[5][2,:]
        self.tensor[5][2,:] = self.tensor[2][2,:]
        self.tensor[2][2,:] = self.tensor[4][2,:]
        self.tensor[4][2,:] = aux
        
    def D_(self):
        aux = torch.clone(self.tensor[0][0,:])
        self.tensor[0][0,:] = self.tensor[4][0,:]
        self.tensor[4][0,:] = self.tensor[2][0,:]
        self.tensor[2][0,:] = self.tensor[5][0,:]
        self.tensor[5][0,:] = aux
        
    def B(self):
        aux = torch.clone(self.tensor[4][:,2])
        self.tensor[4][:,2] = self.tensor[3][:,2]
        self.tensor[3][:,2] = self.tensor[5][:,2]
        self.tensor[5][:,2] = self.tensor[1][:,2]
        self.tensor[1][:,2] = aux
        
    def B_(self):
        aux = torch.clone(self.tensor[4][:,2])
        self.tensor[4][:,2] = self.tensor[1][:,2]
        self.tensor[1][:,2] = self.tensor[5][:,2]
        self.tensor[5][:,2] = self.tensor[3][:,2]
        self.tensor[3][:,2] = aux
    
    def F_(self):
        aux = torch.clone(self.tensor[4][:,0])
        self.tensor[4][:,0] = self.tensor[3][:,0]
        self.tensor[3][:,0] = self.tensor[5][:,0]
        self.tensor[5][:,0] = self.tensor[1][:,0]
        self.tensor[1][:,0] = aux
        
    def F(self):
        aux = torch.clone(self.tensor[4][:,0])
        self.tensor[4][:,0] = self.tensor[1][:,0]
        self.tensor[1][:,0] = self.tensor[5][:,0]
        self.tensor[5][:,0] = self.tensor[3][:,0]
        self.tensor[3][:,0] = aux
        
    def Scramble(self, num=50, get_path=False):
        path = []
        for x in range(num):
            move = rnd.choice((self.R, self.R_, self.L, self.L_, self.U, self.U_, self.B, self.B_, self.F, self.F_, self.D, self.D_))
            path.append(move.__name__)
            move()
        if get_path:
            return path
        else:
            return None
    
    def Restart(self):
        self.tensor = torch.tensor([torch.full([3,3],1).tolist(),
                     torch.full([3,3],2).tolist(),
                     torch.full([3,3],3).tolist(),
                     torch.full([3,3],4).tolist(),
                     torch.full([3,3],5).tolist(),
                     torch.full([3,3],6).tolist()])
        return self.tensor.flatten()
    
    def Sequence(self, moves=None):
        if moves == None:
            print("Please insert a list of moves. Example: ['R','F_','B','U_']")
        else:
            for x in moves:
                exec(f"self.{x}()")
                
    def score(self):
        score = 0
        for x in self.tensor:
            for ind1, y in enumerate(x):
                for ind2, z in enumerate(y):
                    if ind1 != 1 or ind2 != 1:
                        if x[ind1][ind2] == x[1][1]:
                            score+=1
        return score
    
    def action(self):
        pass
    
    def get_next_states(self):
        actions = [self.R, self.R_, self.L, self.L_, self.U, self.U_, self.B, self.B_, self.F, self.F_, self.D, self.D_]
        next_states = {}
        for action in actions:
            initial_state = copy.deepcopy(self.tensor)
            action()
            next_states[action]=self.tensor.flatten().float()
            self.tensor = initial_state
            
        return next_states
    
    def step(self, step, max_steps):
        complete = torch.tensor([torch.full([3,3],1).tolist(),
                     torch.full([3,3],2).tolist(),
                     torch.full([3,3],3).tolist(),
                     torch.full([3,3],4).tolist(),
                     torch.full([3,3],5).tolist(),
                     torch.full([3,3],6).tolist()])
        if step < max_steps or torch.all(torch.eq(self.tensor,complete)):
            done = True
        else:
            done = False
        return self.score(), done


In [5]:
cubo = Cube()

In [6]:
cubo.Sequence(['R','F_'])

In [7]:
cubo.tensor[0]

tensor([[1, 1, 4],
        [1, 1, 4],
        [1, 1, 4]])

In [8]:
rnd.seed(3)

In [9]:
cubo.tensor[1]

tensor([[5, 2, 1],
        [5, 2, 1],
        [5, 2, 1]])

In [10]:
cubo.tensor[2]

tensor([[3, 3, 2],
        [3, 3, 2],
        [3, 3, 2]])

In [11]:
cubo.tensor[3]

tensor([[6, 4, 3],
        [6, 4, 3],
        [6, 4, 3]])

In [12]:
cubo.tensor[4]

tensor([[4, 5, 5],
        [4, 5, 5],
        [4, 5, 5]])

In [13]:
cubo.tensor[5]

tensor([[2, 6, 6],
        [2, 6, 6],
        [2, 6, 6]])

In [6]:
cubo.Scramble()

In [7]:
cubo.tensor[0]

tensor([[1, 5, 6],
        [4, 1, 2],
        [6, 1, 5]])

In [16]:
cubo.Restart()

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6,
        6, 6, 6, 6, 6, 6])

In [8]:
a = cubo.get_next_states()

In [9]:
cubo.step(4,5)

(16, True)

In [10]:
import torch.nn as nn

class DeepQNetwork(nn.Module):
    def __init__(self):
        super(DeepQNetwork, self).__init__()

        self.conv1 = nn.Sequential(nn.Linear(54, 256), nn.LeakyReLU(inplace=True))
        self.conv2 = nn.Sequential(nn.Linear(256, 256), nn.LeakyReLU(inplace=True))
        self.conv3 = nn.Sequential(nn.Linear(256, 256), nn.LeakyReLU(inplace=True))
        self.conv4 = nn.Sequential(nn.Linear(256, 256), nn.LeakyReLU(inplace=True))
        self.conv5 = nn.Sequential(nn.Linear(256, 256), nn.LeakyReLU(inplace=True))
        self.conv6 = nn.Sequential(nn.Linear(256, 1))

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)

        return x

In [11]:
env = Cube()

In [12]:
class opt():
    batch_size = 512
    lr = 3e-4
    gamma = 0.99
    initial_epsilon=1
    final_epsilon=0.005
    num_decay_epochs=5000
    num_epochs=40000
    save_interval=500
    replay_memory_size=3000000
    target_step = 1
    max_steps = 100
    log_path="tensorboard/lr0.003"
    saved_path="trained_models"

In [13]:
def train(opt):
    rnd.seed(3)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(123)
        
    else:
        torch.manual_seed(123)
    if os.path.isdir(opt.log_path):
        shutil.rmtree(opt.log_path)
        
    os.makedirs(opt.log_path)
    
    writer = SummaryWriter(opt.log_path)
    
    model = DeepQNetwork()
    target_model = copy.deepcopy(model)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=opt.lr)
    criterion = nn.MSELoss()
    
    env.Restart()
    env.Scramble()
    state = env.tensor.flatten().float()
    if torch.cuda.is_available():
        model.cuda()
        target_model.cuda()
        state = state.cuda()
    replay_memory = deque(maxlen=opt.replay_memory_size)
    
    epoch = 0
    for x in range(opt.num_epochs):
        step = 0
        while True:
            next_steps = env.get_next_states()
            epsilon = opt.final_epsilon + (max(opt.num_decay_epochs - epoch, 0) * (
                    opt.initial_epsilon - opt.final_epsilon) / opt.num_decay_epochs)
            u = random()
            random_action = u <= epsilon
            next_actions, next_states = zip(*next_steps.items())
            next_states = torch.stack(next_states)
            if torch.cuda.is_available():
                next_states = next_states.cuda()
            model.eval()
            with torch.no_grad():
                predictions = model(next_states)[:, 0]
            if random_action:
                index = randint(0, len(next_steps) - 1)
            else:
                index = torch.argmax(predictions).item()

            next_state = next_states[index, :]
            action = next_actions[index].__name__
            
            next_actions[index]()

            reward, done = env.step(step, opt.max_steps)
            step+=1

            if torch.cuda.is_available():
                next_state = next_state.cuda()
            replay_memory.append([state, reward, next_state, done])
            if done:
                # final_score = env.score
                # final_cleared_lines = env.lines
                env.Restart()
                env.Scramble()
                state = env.tensor.flatten().float()
                if torch.cuda.is_available():
                    state = state.cuda()
                break
            else:
                state = next_state
                continue

        if len(replay_memory) < opt.replay_memory_size / 3333:
            continue
        epoch += 1
        if epoch > 2300:
            opt.batch_size += 1
            if epoch > 3500:
                opt.batch_size += 2
        batch = sample(replay_memory, min(len(replay_memory), opt.batch_size))
        state_batch, reward_batch, next_state_batch, done_batch = zip(*batch)
        state_batch = torch.stack(tuple(state for state in state_batch))
        reward_batch = torch.from_numpy(np.array(reward_batch, dtype=np.float32)[:, None])
        next_state_batch = torch.stack(tuple(state for state in next_state_batch))

        if torch.cuda.is_available():
            state_batch = state_batch.cuda()
            reward_batch = reward_batch.cuda()
            next_state_batch = next_state_batch.cuda()

        q_values = model(state_batch)

        with torch.no_grad():
            next_prediction_batch = target_model(next_state_batch)
        model.train()

        y_batch = torch.cat(
            tuple(reward if done else reward + opt.gamma * prediction for reward, done, prediction in
                  zip(reward_batch, done_batch, next_prediction_batch)))[:, None]


        optimizer.zero_grad()
        loss = criterion(q_values, y_batch)

        writer.add_scalar('Train/Loss', loss.item()
                          # np.array(tbscores[-50:]).mean()
                          , epoch - 1)

        loss.backward()
        optimizer.step()

        final_score = reward

        writer.add_scalar('Train/Score', final_score
                          # np.array(tbscores[-50:]).mean()
                          , epoch - 1)
        # writer.add_scalar('Train/Cleared lines', final_cleared_lines
        #                   # np.array(tbcleared_lines[-50:]).mean()
        #                   , epoch - 1)

        if epoch % opt.target_step == 0:
            target_model.load_state_dict(model.state_dict())
            # target_model = copy.deepcopy(model)
            
        print("Epoch: {}/{}, Action: {}, Score: {}".format(
            epoch,
            opt.num_epochs,
            action,
            final_score))

        # tbscores.append(final_score)
        # tbcleared_lines.append(final_cleared_lines)



        if epoch > 0 and epoch % opt.save_interval == 0:
            torch.save(model, "{}/rubik_{}".format(opt.saved_path, epoch))

        torch.save(model, "{}/rubik".format(opt.saved_path))


In [ ]:
train(opt)

In [ ]:
from vpython import *
import time
fps = 25
faces= {1:(color.white),
        2:(color.blue),
        3:(color.yellow),
        4:(color.green),
        5:(color.red),
        6:(color.orange),
       }
stickers = []
angulo = 0
angulo2 = pi/2


while True:
    for face_color in faces.keys():
        if angulo < 2 * pi:
            for x in (-1, 0, 1):
                for y in (-1, 0, 1):
                    sticker = box(color=faces[cubo.tensor[face_color-1][x+1][y+1].item()], pos=vector(x, y, 1.5),
                                  length=0.98, height=0.98, width=0.05)
                    sticker.rotate(angle=angulo, axis=vector(0,1,0), origin=vector(0, 0, 0))
                    stickers.append(sticker)
            angulo = angulo + pi / 2
        else:
            for x in (-1, 0, 1):
                for y in (-1, 0, 1):

                    sticker = box(color=faces[cubo.tensor[face_color-1][x+1][y+1].item()], pos=vector(x, y, 1.5),
                                  length=0.98, height=0.98, width=0.05)
                    sticker.rotate(angle=angulo2, axis=vector(1,0,0), origin=vector(0, 0, 0))
                    stickers.append(sticker)
            angulo2 =- pi / 2
    time.sleep(20)

In [15]:
test[0]

NameError: name 'test' is not defined

In [ ]:
import torch.nn as nn

class DeepQNetwork(nn.Module):
    def __init__(self):
        super(DeepQNetwork, self).__init__()

        self.conv1 = nn.Sequential(nn.Linear(3*3*6, 128), nn.LeakyReLU(inplace=True))
        self.conv2 = nn.Sequential(nn.Linear(128, 128), nn.LeakyReLU(inplace=True))
        self.conv3 = nn.Sequential(nn.Linear(128, 1))

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)

        return x

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
state = torch.zeros(3 ** 2 * 6, dtype=torch.uint8)
for i in range(6):
    state[3 ** 2 * i: 3 ** 2 * (i + 1)] = i

In [ ]:
state

In [ ]:
state[0:9]

In [ ]:
test = cubo.tensor.flatten()

In [ ]:
test.float()

In [ ]:
model = DeepQNetwork()

In [ ]:
import copy
target = copy.deepcopy(model)

In [ ]:
target(test.float())

In [ ]:
model(test.float())

In [ ]:
if torch.cuda.is_available():
    torch.cuda.manual_seed(123)
else:
    torch.manual_seed(123)
# env = Tetris(width=opt.width, height=opt.height, block_size=opt.block_size)
model = DeepQNetwork()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()
cubo.Restart()
data = cubo.tensor.flatten()
state = data.float()
if torch.cuda.is_available():
    model.cuda()
    state = state.cuda()

replay_memory = deque(maxlen=30000)
epoch = 0
while epoch < opt.num_epochs:
    next_steps = cubo.get_next_states()
    # Exploration or exploitation
    epsilon = opt.final_epsilon + (max(opt.num_decay_epochs - epoch, 0) * (
            opt.initial_epsilon - opt.final_epsilon) / opt.num_decay_epochs)
    u = random()
    random_action = u <= epsilon
    next_actions, next_states = zip(*next_steps.items())
    next_states = torch.stack(next_states)
    if torch.cuda.is_available():
        next_states = next_states.cuda()
    model.eval()
    with torch.no_grad():
        predictions = model(next_states)[:, 0]
    model.train()
    if random_action:
        index = randint(0, len(next_steps) - 1)
    else:
        index = torch.argmax(predictions).item()

    next_state = next_states[index, :]
    action = next_actions[index]

    reward, done = env.step(action, render=False)

    if torch.cuda.is_available():
        next_state = next_state.cuda()
    replay_memory.append([state, reward, next_state, done])
    if done:
        final_score = env.score
        final_tetrominoes = env.tetrominoes
        final_cleared_lines = env.cleared_lines
        state = env.reset()
        if torch.cuda.is_available():
            state = state.cuda()
    else:
        state = next_state
        continue
    if len(replay_memory) < opt.replay_memory_size / 10:
        continue
    epoch += 1
    batch = sample(replay_memory, min(len(replay_memory), opt.batch_size))
    state_batch, reward_batch, next_state_batch, done_batch = zip(*batch)
    state_batch = torch.stack(tuple(state for state in state_batch))
    reward_batch = torch.from_numpy(np.array(reward_batch, dtype=np.float32)[:, None])
    next_state_batch = torch.stack(tuple(state for state in next_state_batch))

    if torch.cuda.is_available():
        state_batch = state_batch.cuda()
        reward_batch = reward_batch.cuda()
        next_state_batch = next_state_batch.cuda()

    q_values = model(state_batch)
    model.eval()
    with torch.no_grad():
        next_prediction_batch = model(next_state_batch)
    model.train()

    y_batch = torch.cat(
        tuple(reward if done else reward + opt.gamma * prediction for reward, done, prediction in
              zip(reward_batch, done_batch, next_prediction_batch)))[:, None]

    optimizer.zero_grad()
    loss = criterion(q_values, y_batch)
    loss.backward()
    optimizer.step()

    print("Epoch: {}/{}, Action: {}, Score: {}, Tetrominoes {}, Cleared lines: {}".format(
        epoch,
        opt.num_epochs,
        action,
        final_score,
        final_tetrominoes,
        final_cleared_lines))
    writer.add_scalar('Train/Score', final_score, epoch - 1)
    writer.add_scalar('Train/Tetrominoes', final_tetrominoes, epoch - 1)
    writer.add_scalar('Train/Cleared lines', final_cleared_lines, epoch - 1)

    if epoch > 0 and epoch % opt.save_interval == 0:
        torch.save(model, "{}/tetris_{}".format(opt.saved_path, epoch))

torch.save(model, "{}/tetris".format(opt.saved_path))

In [ ]:
from PIL import Image

In [ ]:
im = Image.open('WIN_20211124_18_30_05_Pro.jpg')

In [ ]:
im

In [ ]:
test = im.load()

In [ ]:
im.size

In [ ]:
for x in range(im.size[0]):
    for y in range(im.size[1]):
        if 300 < x < 340 and 50 < y < 100:
            test[x,y] = (0,0,0)

In [ ]:
im

In [ ]:
len(test.getdata())

In [ ]:
test_rgb = test.convert('RGB')

In [ ]:
test_rgb.getpixel((320,100))

In [ ]:
test_rgb

In [ ]:
test.size

In [ ]:
640*480